# Spark SQL & Yelp Dataset

In [1]:
# Our usual Spark mantra

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName('My First Spark application') \
    .getOrCreate() 

sc = spark.sparkContext

In [2]:
# Load the JSON files

business = spark.read.json('data/yelp_academic/yelp_academic_dataset_business.json.gz')
checkin = spark.read.json('data/yelp_academic/yelp_academic_dataset_checkin.json.gz')
review = spark.read.json('data/yelp_academic/yelp_academic_dataset_review.json.gz')
tip = spark.read.json('data/yelp_academic/yelp_academic_dataset_tip.json.gz')
user = spark.read.json('data/yelp_academic/yelp_academic_dataset_user.json.gz')

In [3]:
# Create temp views for the DataFrames

business.createOrReplaceTempView("business")
checkin.createOrReplaceTempView("checkin")
tip.createOrReplaceTempView("tip")
review.createOrReplaceTempView("review")
user.createOrReplaceTempView("user")

### How many users have more than 500 fans?

In [4]:
spark.sql('SELECT * FROM user WHERE fans > 500 ').count()

185

### How many businesses from Madison, Wisconsin are represented in the dataset?

In [5]:
spark.sql("SELECT * FROM business WHERE (city ='Madison') AND (state='WI')").count()

3493

### Which US states are represented in the data set? Use full names of states (you will need to look up the list of state abbreviations is you don't know them).

In [6]:
import pandas as pd

In [7]:
states = spark.sql("SELECT DISTINCT state FROM business").toPandas()

In [8]:
convert_states = {"AL":"Alabama","AK":"Alaska","AZ":"Arizona","AR":"Arkansas","CA":"California","CO":"Colorado","CT":"Connecticut","DE":"Delaware","FL":"Florida","GA":"Georgia","HI":"Hawaii","ID":"Idaho","IL":"Illinois","IN":"Indiana","IA":"Iowa","KS":"Kansas","KY":"Kentucky","LA":"Louisiana","ME":"Maine","MD":"Maryland","MA":"Massachusetts","MI":"Michigan","MN":"Minnesota","MS":"Mississippi","MO":"Missouri","MT":"Montana","NE":"Nebraska","NV":"Nevada","NH":"New Hampshire","NJ":"New Jersey","NM":"New Mexico","NY":"New York","NC":"North Carolina","ND":"North Dakota","OH":"Ohio","OK":"Oklahoma","OR":"Oregon","PA":"Pennsylvania","RI":"Rhode Island","SC":"South Carolina","SD":"South Dakota","TN":"Tennessee","TX":"Texas","UT":"Utah","VT":"Vermont","VA":"Virginia","WA":"Washington","WV":"West Virginia","WI":"Wisconsin","WY":"Wyoming"}

In [9]:
states.replace(convert_states,inplace=True)
# this is to only show US states bc there are provinces from other countries
states[states['state'].isin(convert_states.values())] 

,state
0,Arizona
1,South Carolina
3,New Jersey
4,Virginia
7,Nevada
8,Wisconsin
9,California
10,Nebraska
11,Connecticut
12,North Carolina


### What is the text of the funniest review?

In [10]:
fun_df = spark.sql("SELECT text,funny FROM review ORDER BY funny DESC").limit(5).toPandas()

In [11]:
print(fun_df.text[0])

Flew to Arizona a few months ago to try this. Disappointed that this place closed after Gordon Ramsay's return. Their food was one of the best food I had in my whole entire life!! Their caesar salad was great, sauce is very good. Their pizza was THE BEST I HAD IN MY WHOLE ENTIRE LIFE. From the garlic crust to the extremely cheesy topping, the pizza was absolutely outstanding. Wish I could come back but its now closed.


### Which review(s) has/have the most words? What do you notice about the maximum word count?

In [12]:
spark.udf.register('char_len',(lambda x: len(x)))

<function __main__.<lambda>(x)>

In [13]:
q6_query = """
    SELECT text,cool,funny,stars,useful
    FROM review 
    WHERE char_len(text) = (SELECT MAX(char_len(text)) FROM review)
"""

In [14]:
q6_df = spark.sql(q6_query).limit(5).toPandas()

In [15]:
q6_df

,text,cool,funny,stars,useful
0,Overall a good experience. It is pretty expens...,1,0,4.0,0
1,Painted with Oil is a wonderful store with so ...,0,0,5.0,0
2,i'll be honest i go there mainly because of th...,0,0,3.0,0
3,I had high hopes for this shop... Close to the...,0,0,1.0,2
4,I came to Echo Sushi because a good friend of ...,0,0,4.0,0


In [16]:
for text in q6_df.text:
    print(len(text))

999
999
999
999
999


<h4 style='color:green'>The results show us that there is a 999 character limit for reviews.</h4>

### What are the names of the top 10 users who provided the most tips?

<h4>Logic demonstrated in 2 queries</h4>

In [17]:
q7a ="""
    SELECT user_id , count(user_id) tip_count
    FROM tip 
    GROUP BY user_id 
    ORDER BY tip_count DESC
    LIMIT 10
"""

In [18]:
ten_df = spark.sql(q7a).toPandas()

In [19]:
ten_list = tuple(ten_df.user_id[:])
ten_list

('mkbx55W8B8aPLgDqex7qgg',
 'CxDOIDnH8gp9KXzpBHJYXw',
 '6ZC-0LfOAGwaFc5XPke74w',
 '0tvCcnfJnSs55iB6mqPk3w',
 'eZfHm0qI8A_HfvXScwIYsg',
 'O8eDScRAg6ae0l9Bc24uMA',
 '8DGFWco9VeBAxjqsuh1aSw',
 'WJKocp9RE0KatUwh3_DkGg',
 '2EuPAGalYnP7eSxPgFCNDg',
 'QPJJohtGqkMkaN0Gt3TRIg')

In [20]:
q7_query = f"""
    SELECT DISTINCT u.user_id, u.name
        FROM user u
        INNER JOIN  tip t
            ON t.user_id = u.user_id
            WHERE t.user_id IN {ten_list}    
"""

In [21]:
ten_tippers = spark.sql(q7_query).limit(10).toPandas()
ten_tippers

,user_id,name
0,6ZC-0LfOAGwaFc5XPke74w,Samantha
1,O8eDScRAg6ae0l9Bc24uMA,May
2,2EuPAGalYnP7eSxPgFCNDg,Shirley
3,CxDOIDnH8gp9KXzpBHJYXw,Jennifer
4,mkbx55W8B8aPLgDqex7qgg,Momo
5,QPJJohtGqkMkaN0Gt3TRIg,Cherrie
6,WJKocp9RE0KatUwh3_DkGg,Anthony
7,8DGFWco9VeBAxjqsuh1aSw,Kurdy
8,eZfHm0qI8A_HfvXScwIYsg,Christie
9,0tvCcnfJnSs55iB6mqPk3w,Daniel


<h3> Now to roll it into an inner and outer query </h3>

In [22]:
q7_query = """
    SELECT t.user_id , u.name, t.tip_count
        FROM  (SELECT user_id , count(user_id) tip_count FROM tip GROUP BY user_id ORDER BY tip_count DESC) t
        LEFT JOIN  user u
            ON t.user_id = u.user_id  
        ORDER BY t.tip_count DESC
        LIMIT 10
"""

In [23]:
q7_df = spark.sql(q7_query).toPandas()
q7_df

,user_id,name,tip_count
0,mkbx55W8B8aPLgDqex7qgg,Momo,2439
1,CxDOIDnH8gp9KXzpBHJYXw,Jennifer,1598
2,6ZC-0LfOAGwaFc5XPke74w,Samantha,1509
3,0tvCcnfJnSs55iB6mqPk3w,Daniel,1376
4,eZfHm0qI8A_HfvXScwIYsg,Christie,1352
5,O8eDScRAg6ae0l9Bc24uMA,May,1255
6,8DGFWco9VeBAxjqsuh1aSw,Kurdy,1178
7,WJKocp9RE0KatUwh3_DkGg,Anthony,1161
8,2EuPAGalYnP7eSxPgFCNDg,Shirley,1154
9,QPJJohtGqkMkaN0Gt3TRIg,Cherrie,1017


### List the names, number of reviews of businesses in Arizona and total number of reviews of the top 5 users (as determined by who has created the most number of reviews of businesses in Arizona).  Include a column that shows the percentage of reviews that are of businesses from Arizona.

In [24]:
from pyspark.sql.types import IntegerType 
from pyspark.sql.types import FloatType 

In [25]:
q8_query = """
    SELECT user.name AS name, az_bus.o_az_count AS az_count, user.review_count AS total_count,
    (az_bus.o_az_count/user.review_count)*100 as percent
    FROM (
            SELECT r.user_id, count(r.user_id) o_az_count 
            FROM business AS bus 
            RIGHT JOIN review  AS r
            ON bus.business_id = r.business_id
            WHERE bus.state = 'AZ'
            GROUP BY r.user_id
            ORDER BY o_az_count DESC
            ) az_bus
    RIGHT JOIN user
        ON az_bus.user_id = user.user_id
    ORDER BY az_bus.o_az_count DESC
    LIMIT 5
"""

In [26]:
q8_df = spark.sql(q8_query).toPandas()

In [27]:
q8_df

,name,az_count,total_count,percent
0,Brad,1637,1642,99.695493
1,Karen,1559,2340,66.623932
2,Jennifer,1250,1929,64.800415
3,Gabi,1151,1932,59.575569
4,Judy,1059,1193,88.767812
